In [3]:
import fitz  # PyMuPDF
import os

# Function to split PDF into individual pages
def split_pdf_to_images(pdf_path, output_dir):
    doc = fitz.open(pdf_path)
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap()
        output_path = f"{output_dir}/page_{page_num + 1}.png"
        pix.save(output_path)

# Specify paths
pdf_path = '/Users/haydenhenderson/Repos/sig_detector/pdfs/real.pdf'
output_dir = '/Users/haydenhenderson/Repos/sig_detector/real'
os.makedirs(output_dir, exist_ok=True)

# Split PDF
split_pdf_to_images(pdf_path, output_dir)


In [18]:
from PIL import Image
output_dir = '/Users/haydenhenderson/Repos/sig_detector/pages/fake'
# Function to crop edges of an image
def crop_edges(image_path, output_path, crop_size):
    image = Image.open(image_path)
    width, height = image.size
    cropped_image = image.crop((crop_size - 15, crop_size + 35, width - crop_size, height - crop_size))
    cropped_image.save(output_path)

# Crop size in pixels (assuming 1 inch is about 96 pixels)
crop_size = 68
cropped_output_dir = '/Users/haydenhenderson/Repos/sig_detector/f'
os.makedirs(cropped_output_dir, exist_ok=True)

# Crop all pages
for filename in os.listdir(output_dir):
    if filename.endswith(".png"):
        input_path = os.path.join(output_dir, filename)
        output_path = os.path.join(cropped_output_dir, filename)
        crop_edges(input_path, output_path, crop_size)


In [21]:
import cv2
import numpy as np
cropped_output_dir = '/Users/haydenhenderson/Repos/sig_detector/cropped images/r'
# Function to extract individual signatures from a page
def extract_signatures(image_path, output_dir, grid_size=(7, 4)):
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    sig_height = height // grid_size[0]
    sig_width = width // grid_size[1]

    for row in range(grid_size[0]):
        for col in range(grid_size[1]):
            x_start = col * sig_width
            y_start = row * sig_height
            x_end = x_start + sig_width
            y_end = y_start + sig_height
            signature = image[y_start:y_end, x_start:x_end]
            sig_output_path = os.path.join(output_dir, f"{os.path.basename(image_path).split('.')[0]}_sig_{row + 1}_{col + 1}.png")
            cv2.imwrite(sig_output_path, signature)

# Output directory for individual signatures
sig_output_dir = '/Users/haydenhenderson/Repos/sig_detector/real'
os.makedirs(sig_output_dir, exist_ok=True)

# Extract signatures from all cropped pages
for filename in os.listdir(cropped_output_dir):
    if filename.endswith(".png"):
        input_path = os.path.join(cropped_output_dir, filename)
        extract_signatures(input_path, sig_output_dir)


In [26]:
from PIL import Image
import os

# Directory containing the images
input_dir = '/Users/haydenhenderson/Repos/sig_detector/Dataset/real'

# Directory to save the black and white images
output_dir = '/Users/haydenhenderson/Repos/sig_detector/r'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Get a list of all files in the input directory
files = os.listdir(input_dir)

# Iterate through each file
for file in files:
    # Check if the file is a JPEG image
    if file.endswith('.png') or file.endswith('.jpeg'):
        # Open the image
        with Image.open(os.path.join(input_dir, file)) as img:
            # Convert the image to black and white
            bw_img = img.convert('L')
            # Save the black and white image to the output directory
            bw_img.save(os.path.join(output_dir, file)) 